# Task 3
## Часть 1
Нужно написать скрипт, который скачивает все данные прошедших президентских выборов для всех избирательных участков.

Входная точка по ссылке. Затем нужно перейти на сайты региональных избирательных комиссий. Результаты нужно сохранить в cvs-файл, sqlite базе данных или parquet-файле. В итоге должна получиться таблица с полями: - название региона - название ТИК - номер УИК - 20 стандартных полей из итогового протокола

## Часть 2
Нужно, используя Spark:
- найти явку (%) по всем регионам, результат отсортировать по убыванию
- выбрать любимого кандидата и найти тот избиратльный участок, на котором он получил наибольший результат (учитывать участки на которых проголосовало больше 300 человек)
- найти регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна
- посчитать дисперсию по явке для каждого региона (учитывать УИК)
- для каждого кандидата посчитать таблицу: результат (%, округленный до целого) - количество УИК, на которых кандидат получил данный результат

Результаты принимаются в виде Jupyter Notebook, Spark Notebook или исходных файлов на Scala.


## Парсинг сайта
Парсинг происходил с помощью функций в скрипте crawler.py: т.к. сайт вёл себя нестабильно во время парсинга и часто происходили ошибки Timeout, то было принято решение парсить по отдельным регионам, сохраняя результаты в отдельные *.csv документы. А затем с помощью скрипта concat_files.py соединять их воедино.

# Обработка с помощью Spark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType as string, IntegerType as integer
from pyspark import SparkContext
from pyspark.sql import functions as F


In [2]:
spark = SparkSession.builder.getOrCreate()

## Определение полей файла

In [3]:
f_name = 'all_csv.csv'

schema= StructType([
            StructField("index1",                      integer(), False), #1
            StructField("index2",                      integer(), False), #2
            StructField("region_name",                 string(), False), #3
            StructField("tik_name",                    string(), False), #4
            StructField("uik_num",                     string(), False), #5
            StructField("cnt_list_voters",             integer(), False),#6
            StructField("bulletins_received_comm",     integer(), False),#7
            StructField("gived_bulletins_ahead",       integer(), False),#8
            StructField("inside_bulletins",            integer(), False),#9
            StructField("outside_bulletins",           integer(), False),#10
            StructField("cancel_bulletins",            integer(), False),#11
            StructField("transfer_box_bulletins",      integer(), False),#12
            StructField("stat_box_bulletins",          integer(), False),#13
            StructField("invalid_bulletins",           integer(), False),#14
            StructField("valid_bulletins",             integer(), False),#15
            StructField("lost_bulletins",              integer(), False),#16
            StructField("uncount_bulletins",           integer(), False),#17
            StructField("baburin",                     integer(), False),#18
            StructField("grudinin",                    integer(), False),#19
            StructField("jirinovskiy",                 integer(), False),#20
            StructField("putin",                       integer(), False),#21
            StructField("sobchak",                     integer(), False),#22
            StructField("suraykin",                    integer(), False),#23
            StructField("titov",                       integer(), False),#24
            StructField("yavlinskiy",                  integer(), False)]#25
        )

data = spark.read.csv(f_name, schema=schema)

## Найти явку (%) по всем регионам, результат отсортировать по убыванию

In [4]:
data.groupBy('region_name').agg(
    F.sum('cnt_list_voters').alias('cnt_list_voters'),
    F.sum('stat_box_bulletins').alias('stat_box_bulletins'),
    F.sum('transfer_box_bulletins').alias('transfer_box_bulletins'),
).select(
    F.col('region_name'),
    ((F.col('stat_box_bulletins') + F.col('transfer_box_bulletins')) / F.col('cnt_list_voters') * 100).alias('presence, %')
).sort('presence, %', ascending=False).show(truncate=False)

+-----------------------------------+-----------------+
|region_name                        |presence, %      |
+-----------------------------------+-----------------+
|Республика Тыва                    |93.62542974951742|
|Ямало-Ненецкий автономный округ    |91.87213306617981|
|Кабардино-Балкарская Республика    |91.6978754085207 |
|Чеченская Республика               |91.50929703298173|
|Республика Северная Осетия - Алания|89.93704831730909|
|Республика Дагестан                |87.44351419820495|
|Карачаево-Черкесская Республика    |87.343809390099  |
|Кемеровская область                |83.07556958224816|
|Чукотский автономный округ         |82.26051697921946|
|Республика Ингушетия               |81.95937165135314|
|Брянская область                   |79.65976807571519|
|Тюменская область                  |78.8829673763749 |
|Республика Мордовия                |77.83648793754097|
|Краснодарский край                 |77.83466166202626|
|Республика Татарстан (Татарстан)   |77.36526548

## Выбрать любимого кандидата и найти тот избиратльный участок, на котором он получил наибольший результат (учитывать участки на которых проголосовало больше 300 человек)

In [5]:
from numpy.random import randint
fav_candid = schema[17 + randint(0, 8)].name
print(fav_candid)
data.select(
    F.col('region_name'),
    F.col('tik_name'),
    F.col('uik_num'),
    (F.col(fav_candid) / F.col('cnt_list_voters') * 100).alias('percentage, %')
).sort('percentage, %', ascending=False).show(1, truncate=False)

grudinin
+---------------+------------------------+-------+-------------+
|region_name    |tik_name                |uik_num|percentage, %|
+---------------+------------------------+-------+-------------+
|Приморский край|Владивосток, Фрунзенская|5924   |80.0         |
+---------------+------------------------+-------+-------------+
only showing top 1 row



## Найти регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна

In [6]:
data.select(
    F.col('region_name'),
    F.col('tik_name'),
    F.col('cnt_list_voters'),
    F.col('stat_box_bulletins'),
    F.col('transfer_box_bulletins')
).groupBy('region_name', 'tik_name').agg(
    F.sum('cnt_list_voters').alias('cnt_list_voters'),
    F.sum('stat_box_bulletins').alias('stat_box_bulletins'),
    F.sum('transfer_box_bulletins').alias('transfer_box_bulletins'),
).select(
    F.col('region_name'), ((F.col('stat_box_bulletins') + F.col('transfer_box_bulletins')) / F.col('cnt_list_voters')).alias('presence')
).groupBy('region_name').agg(F.max('presence').alias('max_presence'), F.min('presence').alias('min_presence')
).select(
    F.col('region_name'), ((F.col('max_presence') - F.col('min_presence')) * 100).alias('difference_presence, %')
).sort('difference_presence, %', ascending=False).show(1, truncate=False)

+---------------------+----------------------+
|region_name          |difference_presence, %|
+---------------------+----------------------+
|Архангельская область|49.8591648406849      |
+---------------------+----------------------+
only showing top 1 row



## Посчитать дисперсию по явке для каждого региона (учитывать УИК)

In [7]:
data.groupBy('region_name').agg(
    F.stddev( (F.col('stat_box_bulletins') + F.col('transfer_box_bulletins')) / F.col('cnt_list_voters') ).alias('std_dev_presence')
).sort('std_dev_presence', ascending=False).show(truncate=False)

+--------------------------------+-------------------+
|region_name                     |std_dev_presence   |
+--------------------------------+-------------------+
|Сахалинская область             |0.20188798623588156|
|Камчатский край                 |0.16873475777512123|
|Мурманская область              |0.16742131453895345|
|Приморский край                 |0.16639015831512335|
|Республика Адыгея (Адыгея)      |0.16333249598843566|
|Самарская область               |0.1562883627711599 |
|Саратовская область             |0.15426531267153373|
|Республика Калмыкия             |0.15350844523090657|
|Воронежская область             |0.14975754071870498|
|Магаданская область             |0.14387651017723133|
|Республика Татарстан (Татарстан)|0.14172353736111934|
|Липецкая область                |0.14096365252591136|
|Архангельская область           |0.1409455295612833 |
|Краснодарский край              |0.1394657822552809 |
|Хабаровский край                |0.13811595897262202|
|Белгородс

## Для каждого кандидата посчитать таблицу: результат (%, округленный до целого) - количество УИК, на которых кандидат получил данный результат

In [8]:
candidates = [_.name for _ in schema[17:]]

result_candidates = [(candidate,
data.select(
    (F.ceil(F.col(candidate) / F.col('cnt_list_voters') * 100)).alias('result, %')
    ).groupBy('result, %').agg(F.count('result, %')
    ).sort('count(result, %)', ascending=False))
    for candidate in candidates]

In [9]:
for candidate in result_candidates:
    name, frame = candidate
    print(name)
    frame.show()

baburin
+---------+----------------+
|result, %|count(result, %)|
+---------+----------------+
|        1|           72642|
|        0|           19436|
|        2|            4391|
|        3|             504|
|        4|             153|
|        5|              70|
|        6|              26|
|        7|              20|
|        8|              17|
|       10|               9|
|        9|               7|
|       11|               4|
|       15|               3|
|       12|               2|
|       20|               2|
|       17|               2|
|       22|               1|
|       24|               1|
|       19|               1|
|       13|               1|
+---------+----------------+
only showing top 20 rows

grudinin
+---------+----------------+
|result, %|count(result, %)|
+---------+----------------+
|        8|           12848|
|        7|           12047|
|        9|           10891|
|        6|            9421|
|       10|            7978|
|        5|            6978|
